## 0. Import packages

In [73]:
import pandas as pd
import gzip
import math
from tqdm import tqdm
from itertools import chain, combinations
from Association import Association
tqdm.pandas() #for progres_apply etc.

## 1. Load data from previous step

In [74]:
import pickle
rules = pickle.load(open("pickle_dumps/article_rules.p", "rb"))

In [75]:
print(len(rules))
for i in range(10):
    if i < len(rules):
        print(rules[i])

7566
Conf: 0.96	Supp: 26.00	 {1025, 2954, 132} => {7}
Conf: 0.95	Supp: 20.00	 {562, 20396, 23214} => {7}
Conf: 0.93	Supp: 27.00	 {1025, 3, 132} => {7}
Conf: 0.92	Supp: 34.00	 {1025, 171, 132} => {7}
Conf: 0.92	Supp: 23.00	 {2219, 14364, 7} => {1025}
Conf: 0.92	Supp: 23.00	 {1025, 3, 2219} => {7}
Conf: 0.92	Supp: 23.00	 {3, 2219, 7} => {1025}
Conf: 0.92	Supp: 23.00	 {1025, 3, 3327} => {7}
Conf: 0.92	Supp: 22.00	 {129, 2219, 1025} => {7}
Conf: 0.91	Supp: 21.00	 {1025, 7, 23047} => {132}


In [76]:
class Recommender():
    def __init__(self, df_train, assos, df_test):
        pass
    
    def recommend(self, user_id, k):
        pass        

In [77]:
class Assos_Recommender(Recommender):
    def __init__(self, assos):
        # use popular items to fill unused space
        # TODO (df_train is used for this)
        
        self.assos = assos
        self.df_test = pickle.load(open("pickle_dumps/test_df.p", "rb"))
        self.df_test = self.df_test.set_index("user_id").to_dict()["item_id"]
        
    def get_items(self, user_id):
        return self.df_test[user_id]
    
    def recommend(self, user_id, k):
        user_items = self.get_items(user_id)
        recommendation = set()
        for a in self.assos:
            for l in a.left:
                # break if any element is not in the user session
                if l not in user_items:
                    break
                    
                # all elements are in the user session --> add RHS of rule to the recommendation
                for item in a.right:
                    r = item
                    if r not in user_items and (len(recommendation) < k or k == -1):
                        recommendation.add(r)
                        
            if len(recommendation) >= k and k != -1:
                break
                
        return recommendation

In [81]:
def hitrate(rc, k):
    hits = 0
    counts = 0
    avg_len = 0
    for user in tqdm(rc.df_test):
        r = rc.recommend(user, k)
        avg_len += len(r)
        for item in rc.df_test[user]:
            hits += item in r
            counts += 1
    
    print("avg len: {:f}".format(avg_len / len(rc.df_test)))
    if k == -1:
        print("HR\t{:.2f}%".format((hits / counts) * 100))
    else:
        print("HR@{}\t{:.2f}%".format(k, (hits / counts) * 100))
    return hits / counts

In [ ]:
rc = Assos_Recommender(rules)
hr = hitrate(rc, -1)

100%|█████████▉| 296861/297122 [04:49<00:00, 991.66it/s] 

In [ ]:
print(hr)

# ??. Store the association rules as pickle file for further use

In [80]:
# import pickle
# pickle.dump(rules, open("pickle_dumps/article_rules.p", "wb"))